In [1]:
import importlib
# import pdf2text as p2t
# importlib.reload(p2t)
import subprocess
import json
from rdkit import Chem
from indigo import Indigo
indigo = Indigo()
import re
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 5000)
pd.set_option('display.max_rows', 1500)
import reactions_converter as rc
importlib.reload(rc)
import copy
from itertools import permutations, combinations  
import convert_to_docx as ctd
from tqdm import tqdm

In [ ]:
#target structure
reaction_dict = {
                                                                                                                                    'doi': '', #дои статьи, откуда оно взялось
    'reaction_rxn': '', #реакция в формате rxn
    'products': [{'molfile': '',
                  'amount': 0, #in mmol
                  'weight': 0, #in g
                  'vol': 0, #in mL
                  'yield': 0 #in %
                 }], #список словарей для нескольких продуктов
    'reagents': [{'molfile': '',
                  'role': '', #reactant, reagent, catalyst
                  'amount': 0, #in mmol
                  'weight': 0, #in g
                  'vol': 0, #in mL
                 }],
    'scale': 0, # = максимальный amount среди продуктов
    'temperature': [0], #темпеатура в градусах Цельсия. Список так как может быть несколько температур
    'time': [0], #время реакции в часах. Список так как может быть несколько этапов
    'misc': [''], #список строк со всякими остальными условиями '20 mA', '450 nm 400 W'
    'solvent': [{'solvent_name': '', 'solvent_vol': 0}], #список словарей, где указаны объемы и названия растворителей
    'protocol': '' #сюда тупо строкой падает методика
}

# Copy parsed protocols

In [ ]:
#copy files from parsed
mistral_folder = '/media/oleg/second_ssd/rxn_parsing_data/processed'
main_folder = '/media/oleg/second_ssd/rxn_parsing_data'
for parse_id in tqdm(os.listdir(mistral_folder)):
    #join splitted rxns
    temp_res = []
    for num_parts in os.listdir(os.path.join(mistral_folder, parse_id)):
        with open(os.path.join(mistral_folder, parse_id, num_parts, 'reactions_parsed.json'), 
                  encoding = 'utf-8') as f:
            temp_res += json.loads(f.read())
    with open(os.path.join(main_folder, parse_id, 'reactions_parsed.json'),  'w', encoding = 'utf-8') as f:
        f.write(json.dumps(temp_res))

# Save to docx


In [ ]:
importlib.reload(ctd)

In [ ]:
from docx import Document
from tqdm import tqdm

In [ ]:
document = Document()
for folder in tqdm(os.listdir('/media/oleg/second_ssd/temp_files_parser')):
    folder_path = os.path.join('/media/oleg/second_ssd/temp_files_parser', folder)
    if os.path.isdir(folder_path):
        with open(os.path.join(folder_path, 'reactions_final.json')) as f:
            reactions = json.loads(f.read())
        ctd.get_doc_reactions(reactions, folder, document)


In [ ]:
document.save('/media/oleg/second_ssd/temp_files_parser/total_reactions_report.docx')

# Split total dataset to portions

In [ ]:
folder_list = os.listdir('/media/oleg/hard_for_data/papers_for_parse/orglett/orglett_full')

In [ ]:
len(folder_list)

In [ ]:
num_batches = round(len(folder_list)/300)

In [ ]:
target_folder = '/media/oleg/second_ssd/rxn_parsing_data'
for idx in range(num_batches+1):
    sample = folder_list[idx*300: (idx+1)*300]
    with open(os.path.join(target_folder, str(idx)+'_input.json'), 'w') as f:
        f.write(json.dumps(sample))
    print(len(sample))

In [ ]:
idx_list = [idx for idx in range(num_batches+1)]


In [ ]:
idx_list

In [ ]:
with open('/media/oleg/second_ssd/rxn_parsing_data/file_index.txt', 'w') as f:
    f.writelines([str(x) + '\n' for x in  idx_list[1:]])

In [ ]:
/media/oleg/second_ssd/rxn_parsing_data/0/10.1021_ol9024309/

In [ ]:
temp_folder = '/media/oleg/second_ssd/rxn_parsing_data/0'

# Copy files to server HSE

In [ ]:
parse_ids = range(34, 100)

In [ ]:
target_files = [os.path.join('/media/oleg/second_ssd/rxn_parsing_data', str(x) + '_input.json') for x in parse_ids]

In [ ]:
target_files

In [ ]:
folder_list = []
for inp_file in target_files:
    with open(inp_file) as f:
        folder_list += json.loads(f.read())

In [ ]:
len(folder_list)

In [ ]:
global_folder = '/media/oleg/hard_for_data/papers_for_parse/orglett/orglett_full'

In [ ]:
folder_list = [os.path.join(global_folder, x) for x in folder_list ]

In [ ]:
with open('/media/oleg/second_ssd/rxn_parsing_data/34_99_inputs.txt', 'w') as f:
    f.writelines([fname+'\n' for fname in folder_list])

In [ ]:
for idx in range(126, 127):
    if idx %2 == 0:
        parse_ids = [str(x) for x in [idx,idx+1]]
        string = f"""#! /bin/bash
#SBATCH --job-name="{'_'.join(parse_ids)}_data_prep"
#SBATCH --gpus=1
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --time=0-24:0
#SBATCH --output="stdout.%j.txt"%j.out
#SBATCH --error="stderr.%j.txt"%j.out
module purge
module load Python/Anaconda_v11.2021

source deactivate
source activate molscribe

nvidia-smi

global_folder="/home/dchusov/inp_data/orglett"
result_folder="/home/dchusov/temp_parsing_data"

for input_idx in {' '.join(parse_ids)}
do
temp_folder=$result_folder/$input_idx
echo "Start processing " 
echo "creating temp folders and making pngs and texts" 
mkdir $temp_folder
python /home/dchusov/reaction_parser/create_text_image_mp.py $global_folder $temp_folder &
sleep 1000
echo "running opsin in parallel with creation of text and images"
python /home/dchusov/reaction_parser/running_opsing.py $temp_folder &
sleep 40
echo "running mols recognition"
python /home/dchusov/ocsr_molscribe_mp/recognize_mols.py $temp_folder
done"""
        with open(os.path.join('/media/oleg/second_ssd/rxn_parsing_data/sbatches_for_ocr', 
                               '_'.join(parse_ids) + '_ocr.sbatch'), 'w') as f:
            f.write(string)

In [ ]:
result_folder="/home/dchusov/temp_parsing_data"

for input_idx in 34 46 47 67 72 73 74 90 91 92 93 94 95 96 97 98 99 100 101 117 126
do
temp_folder=$result_folder/$input_idx
echo "process $input_idx" 
zip -r $input_idx".zip" $temp_folder
done

In [ ]:
string = """#! /bin/bash
#SBATCH --job-name="35_data_prep"
#SBATCH --gpus=1
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --time=0-24:0
#SBATCH --output="stdout.%j.txt"%j.out
#SBATCH --error="stderr.%j.txt"%j.out
module purge
module load Python/Anaconda_v11.2021

source deactivate
source activate molscribe

nvidia-smi
global_folder="/home/dchusov/inp_data/orglett"
result_folder="/home/dchusov/temp_parsing_data"

input_idx=34
temp_folder=$result_folder/$input_idx
echo "Start processing" 
echo "creating temp folders and making pngs and texts" 
mkdir $temp_folder
python /home/dchusov/reaction_parser/create_text_image_mp.py $global_folder $temp_folder &
sleep 1000
echo "running opsin in parallel with creation of text and images"
python /home/dchusov/reaction_parser/running_opsing.py $temp_folder &
sleep 40
echo "running mols recognition"
python /home/dchusov/ocsr_molscribe_mp/recognize_mols.py $temp_folder
"""
print(string)

with open(os.path.join('/media/oleg/second_ssd/rxn_parsing_data/sbatches_for_ocr', 
                       '35_ocr.sbatch'), 'w') as f:
    f.write(string)

# Check full processing 

In [ ]:
#is df_subs_mols.csv exists
folder = '/media/oleg/second_ssd/rxn_parsing_data'
for file in os.listdir(folder):
    if os.path.isdir(os.path.join(folder,    file)):
        if not os.path.exists(os.path.join(folder, file, 'df_subs_mols.csv')):
            print(file)

In [ ]:
#is df_subs_mols.csv exists
folder = '/media/oleg/second_ssd/rxn_parsing_data'
for file in os.listdir(folder):
    if os.path.isdir(os.path.join(folder,    file)):
        if not os.path.exists(os.path.join(folder, file, file+ '_1')):
            print(file)

In [ ]:
#is reactions_parsed.json exists
folder = '/media/oleg/second_ssd/rxn_parsing_data/processed'
for parse_id in os.listdir(folder):
    if os.path.isdir(os.path.join(folder, parse_id)):
        for subfolder in os.listdir(os.path.join(folder, parse_id)):
            if not os.path.exists(os.path.join(folder, parse_id, subfolder, 'reactions_parsed.json')):
                print(parse_id)

In [ ]:
#find rest folders
folder = '/media/oleg/second_ssd/rxn_parsing_data'
for idx in range(0, 127):
    if not os.path.exists(os.path.join(folder, str(idx))):
        print(idx)

In [ ]:
#check processed
folder = '/media/oleg/second_ssd/rxn_parsing_data/processed'
for idx in range(0, 127):
    if not os.path.exists(os.path.join(folder, str(idx))):
        print(idx)

# Creating reactions

In [ ]:
import json 
from sys import argv
import os

from rdkit import Chem
from indigo import Indigo
indigo = Indigo()
import re

import pandas as pd

import reactions_converter as rc

import copy
from itertools import permutations, combinations  
import warnings
warnings.filterwarnings('ignore')

In [ ]:
importlib.reload(rc)

In [ ]:
main_folder = '/media/oleg/second_ssd/rxn_parsing_data'
for parse_id in tqdm(os.listdir(main_folder)):
    if not os.path.isdir(os.path.join(main_folder, parse_id)):
        continue
    if not os.path.exists(os.path.join(main_folder, parse_id, 'reactions_parsed.json')):
        continue
        
    global_folder = os.path.join(main_folder, parse_id)
    
    # folder_name = '10.1039_c8ob02305k'
    for folder_name in os.listdir(global_folder):
        folder = os.path.join(global_folder, folder_name)
        if not os.path.isdir(folder):
            continue
        if not os.path.exists(os.path.join(folder, 'protocols.json')):
            continue
        if os.path.exists(os.path.join(folder, 'bad_reactions_final.json')):
            continue  #already done
        

        with open(os.path.join(folder, 'joint_sorted_dict.json')) as f:
            joint_sorted_dict = json.loads(f.read())
        with open(os.path.join(folder, 'compid_descr.json')) as f:
            compid_descr = json.loads(f.read()) 
        with open(os.path.join(folder, 'code2compid.json')) as f:
            code2compid = json.loads(f.read()) 
        with open(os.path.join(folder, 'compid2name.json')) as f:
            compid_to_name = json.loads(f.read()) 
        with open(os.path.join(global_folder, 'reactions_parsed.json')) as f:
            protocols = json.loads(f.read())
        protocols = [x for x in protocols if x['init_paper'] == folder_name]
        
        #clean comp dicts
        compid_to_del = []
        for compid, smiles in compid_descr.items():
            try:
                mol = indigo.loadMolecule(smiles)
                mwt = mol.molecularWeight()
            except:
                compid_to_del.append(compid)        
        
        for compid in compid_to_del:
            compid_descr.pop(compid, None)
            compid_to_name.pop(compid, None)
            codes2del = []
            for code in code2compid:
                if code2compid[code] == compid:
                    codes2del.append(code)
            for code in codes2del:
                code2compid.pop(code, None)
            name2del = []
            for name in joint_sorted_dict:
                if joint_sorted_dict[name] == compid:
                    name2del.append(name)
            for name in name2del:
                joint_sorted_dict.pop(name, None)
        
        
        
        print('Initial of protocols for ', parse_id, folder_name, ':', len(protocols))
        transformed_protocols = []
        for protocol in protocols:
            try:
                transformed_protocols += rc._transform_reaction_dict(protocol) #extract reagents names and reorganize naming
            except Exception as e:
                pass #print(e, '\n', protocol)
        for idx, item in enumerate(transformed_protocols):
            item['is_general'] = rc._is_general(item['procedure'], item['procedure_name'])
            item['id'] = idx
        gen_prot_dict = {}
        for item in transformed_protocols:
            item['refers_to'] = []
            if item['is_general']:
                for name in list(set(item['procedure_name'])):
                    gen_prot_dict[name] = item['id']
            else:
                for gp_name in gen_prot_dict:
                    if gp_name.replace(' ', '') in item['procedure'].replace(' ', ''):
                        item['refers_to'].append(gen_prot_dict[gp_name])

        reactions = []
        gen_proc_cond = {} # {idx: {'conditions': conditions, 'protocol': procedure}}

        used_idx = []
        for reaction in transformed_protocols:
            reaction_created = False

            if reaction['is_general']:
                #в этом случае из протокола сохраняем отдельно условия, MiscInch и растворители
                misc_solv_list = []
                for comp in reaction['compounds']:
                    if comp['compound_role'] != 'target': 
                        misc_solv_list.append(comp)

                gp_ent = {'procedure': reaction['procedure'],
                          'conditions': reaction['conditions'],
                          'misc_solv_list': misc_solv_list } #=solv_gen
                gen_proc_cond[reaction['id']] = gp_ent

            elif len(reaction['refers_to']) > 0:
                #дополняем реакцию условиями из общей методики
                for gen_proc_id in reaction['refers_to']:
                    reaction['conditions'] += gen_proc_cond[gen_proc_id]['conditions']

                    #check if there are compounds in gen_proc duplicate already mentioned compounds in reaction
                    existing_comps_with_amounts = []
                    for comp in reaction['compounds']:
                        if 'amounts' not in comp:
                            comp['amounts'] = []
                        if len(comp['amounts']) > 0:
                            existing_comps_with_amounts.append(comp['compound_id'])
                    appending_comps = []
                    for comp in gen_proc_cond[gen_proc_id]['misc_solv_list']:
                        if comp['compound_id'] not in existing_comps_with_amounts:
                            appending_comps.append(comp)

                    reaction['compounds'] = appending_comps + reaction['compounds'] #ставим общие условия в начале            
            reaction_smi = rc._transform_reaction_combs(reaction, compid_descr, code2compid, compid_to_name)

            reaction_smi['initial_protocol'] = reaction['procedure']
            reaction_smi['used_idx'] = reaction['idx'] if type(reaction['idx']) == list else [reaction['idx']]
            reactions.append(reaction_smi)

        used_idx = []
        final_rxn_list = []
        for reaction in reactions:
            if rc._is_true_reaction(reaction):
                already_added = False
                for idx in reaction['used_idx']:
                    if idx in used_idx:
                        already_added = True

                if not already_added:
                    final_rxn_list.append(reaction)
                    used_idx += reaction['used_idx']

        for reaction in reactions:
            if rc._is_true_product(reaction):
                already_added = False
                for idx in reaction['used_idx']:
                    if idx in used_idx:
                        already_added = True

                if not already_added:
                    final_rxn_list.append(reaction)
                    used_idx += reaction['used_idx']
        print('Adequate reactions found', len(final_rxn_list))
        
        #saving all other reactions
        bad_reactions = []
        for reaction in reactions:
            already_added = False
            for idx in reaction['used_idx']:
                if idx in used_idx:
                    already_added = True        
            if not already_added:
                bad_reactions.append(reaction)
                used_idx += reaction['used_idx']
        
        for rxn in final_rxn_list:
            rxn.pop('rxn', None) 
        for rxn in bad_reactions:
            rxn.pop('rxn', None)         
        print(folder_name, ':', len(bad_reactions), 'bad reactions', '\n')
        with open(os.path.join(folder, 'reactions_final.json'), 'w', encoding = 'utf-8') as f:
            f.write(json.dumps(final_rxn_list))
        with open(os.path.join(folder, 'bad_reactions_final.json'), 'w', encoding = 'utf-8') as f:
            f.write(json.dumps(bad_reactions))

# Merging reactions thorughout the batch

In [9]:
total_good_rxn = []
total_bad_rxn = []
folder='/media/oleg/second_ssd/rxn_parsing_data'
for parse_id in tqdm(range(127)):
    for subfolder in os.listdir(os.path.join(folder, str(parse_id))):
        if os.path.exists(os.path.join(folder, str(parse_id), subfolder, 'reactions_final.json')):
            with open(os.path.join(folder, str(parse_id), subfolder, 'reactions_final.json')) as f:
                new_good_rxns = json.loads(f.read())
                for item in new_good_rxns:
                    item['ref'] = subfolder
                total_good_rxn += new_good_rxns
            with open(os.path.join(folder, str(parse_id), subfolder, 'bad_reactions_final.json')) as f:
                new_bad_rxns = json.loads(f.read())
                for item in new_bad_rxns:
                    item['ref'] = subfolder
                total_bad_rxn += new_bad_rxns        
        

100%|█████████████████████████████████████████| 127/127 [04:18<00:00,  2.03s/it]


In [13]:
with open('/media/oleg/second_ssd/rxn_parsing_data/good_rxns.json', 'w') as f:
    f.write(json.dumps(total_good_rxn))

In [14]:
with open('/media/oleg/second_ssd/rxn_parsing_data/bad_rxns.json', 'w') as f:
    f.write(json.dumps(total_bad_rxn))

In [10]:
len(total_bad_rxn)

860896

In [11]:
len(total_good_rxn)

487503

In [12]:
total_good_rxn[100]

{'procedure': ' (Angew. Chem. Chem. 2: A solution of  4-bromobenzonitrile  (0.91 g, 5 mmol),  DIPEA  (2 mL, 15 mmol),  Pd(OAc)2  (25 mg, 2 mol%), DPEDhos (65 mg, 2.4 mol%) and  n-Butyl vinyl ether  in  BuOH  was stirred at 95 oC for 1 h. The solution was evaporated under reduced pressure. The residue was purified by flash column chromatography on silica_gel ( hexane  /  AcOEt  = 200:1 to 100:1) to afford 0.54 g (2.7 mmol, 54%) of 1u as a colorless oil. ',
 'conditions': [{'temperature': 95.0}, {'time': 1.0}],
 'compounds': [{'score': 1.5499999999999998,
   'smiles': 'BrC1=CC=C(C#N)C=C1',
   'compound_role': 'REACTANT',
   'digit_amounts': {'mass': 910.0, 'amount': 5.0, 'mol%': 100.0}},
  {'score': 1.0499999999999998,
   'smiles': 'C(C)(C)N(CC)C(C)C',
   'compound_role': 'REACTANT',
   'digit_amounts': {'volume': 2.0,
    'amount': 15.0,
    'mol%': 300.0,
    'mass': 1938.0}},
  {'score': -0.3,
   'smiles': 'InChI=1S/2C2H4O2.Pd/c2*1-2(3)4;/h2*1H3,(H,3,4);/q;;+2/p-2',
   'compound_role'

In [ ]:
importlib.reload(ctd)

In [ ]:
folder='/media/oleg/second_ssd/rxn_parsing_data'
doc_idx = 0
document_good = Document()
document_bad = Document()
curr_len = 0

for parse_id in tqdm(range(127)):
    for subfolder in os.listdir(os.path.join(folder, str(parse_id))):
        if os.path.exists(os.path.join(folder, str(parse_id), subfolder, 'reactions_final.json')):
            with open(os.path.join(folder, str(parse_id), subfolder, 'reactions_final.json'), encoding = 'utf-8') as f:
                temp_good_rxn = json.loads(f.read())
            ctd.get_doc_reactions(temp_good_rxn, subfolder, document_good)
            curr_len+= len(temp_good_rxn )
            with open(os.path.join(folder, str(parse_id), subfolder, 'bad_reactions_final.json'), encoding = 'utf-8') as f:
                temp_bad_rxn = json.loads(f.read())         
            ctd.get_doc_reactions(temp_bad_rxn, subfolder, document_bad)
        if curr_len > 5000:
            curr_len = 0
            document_good.save(os.path.join(folder, str(doc_idx) + '_good.docx'))
            document_bad.save(os.path.join(folder, str(doc_idx) + '_bad.docx')) 
            doc_idx += 1
            document_good = Document()
            document_bad = Document()

In [ ]:
total_good_rxn[0]

In [ ]:
import convert_to_docx as ctd
from docx import Document

In [ ]:
document = Document()

In [ ]:
ctd.get_doc_reactions(total_good_rxn[:10], 'doi', document)

In [ ]:
document.save('test.doc')